In [ ]:
#%% [markdown]
# # Producto Computacional 3: Modelamiento Predictivo con Datos MHEALTH
# 
# **Nombre:** <TU_NOMBRE_AQUÍ>
# 
# ## Objetivo General
# 
# Aplicar técnicas de Machine Learning para construir y validar modelos capaces de reconocer actividades humanas a partir de los sensores del dataset MHEALTH, evaluando su desempeño mediante métricas apropiadas.
# 
# ## Modelos a Implementar
# 
# 1.  **Random Forest Classifier:** Un modelo de ensamble robusto que no requiere escalado de datos.
# 2.  **K-Nearest Neighbors (KNN):** Un modelo basado en distancia que sí requiere escalado de datos, permitiéndonos cumplir con ese requerimiento.

#%%
# ------------------------------------------------
# 0. IMPORTACIÓN DE LIBRERÍAS
# ------------------------------------------------
import os
import glob
import zipfile
import urllib.request
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Configuraciones de visualización
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style('whitegrid')

print("Librerías importadas exitosamente.")

#%% [markdown]
# ## 1. Carga y Preprocesamiento de los Datos
# 
# ### 1.1. Descarga y Descompresión de los Datos
# 
# El dataset MHEALTH se descargará del repositorio UCI. Consiste en 10 archivos `.log` (uno por sujeto).

#%%
# ------------------------------------------------
# 1.1. DESCARGA Y EXTRACCIÓN DE DATOS
# ------------------------------------------------
DATA_URL = "http://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip"
ZIP_PATH = "MHEALTHDATASET.zip"
DATA_DIR = "MHEALTHDATASET"

# Descargar el archivo si no existe
if not os.path.exists(ZIP_PATH):
    print(f"Descargando {ZIP_PATH}...")
    urllib.request.urlretrieve(DATA_URL, ZIP_PATH)
    print("Descarga completa.")
else:
    print(f"{ZIP_PATH} ya existe.")

# Descomprimir el archivo si el directorio no existe
if not os.path.exists(DATA_DIR):
    print(f"Descomprimiendo {ZIP_PATH}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(".")
    print(f"Archivos extraídos en el directorio '{DATA_DIR}'.")
else:
    print(f"El directorio '{DATA_DIR}' ya existe.")

#%% [markdown]
# ### 1.2. Carga y Consolidación de los Datos
# 
# Se leerán los 10 archivos `.log` y se consolidarán en un único DataFrame de Pandas. Se definirán las columnas y las etiquetas de actividad según el `README.txt` del dataset.

#%%
# ------------------------------------------------
# 1.2. DEFINICIÓN DE ESTRUCTURA Y CARGA
# ------------------------------------------------

# Definir los nombres de las columnas (23 sensores + 1 etiqueta)
column_names = [
    'Chest_Accel_X', 'Chest_Accel_Y', 'Chest_Accel_Z',
    'Chest_ECG_Lead1', 'Chest_ECG_Lead2',
    'Ankle_Accel_X', 'Ankle_Accel_Y', 'Ankle_Accel_Z',
    'Ankle_Gyro_X', 'Ankle_Gyro_Y', 'Ankle_Gyro_Z',
    'Ankle_Mag_X', 'Ankle_Mag_Y', 'Ankle_Mag_Z',
    'Arm_Accel_X', 'Arm_Accel_Y', 'Arm_Accel_Z',
    'Arm_Gyro_X', 'Arm_Gyro_Y', 'Arm_Gyro_Z',
    'Arm_Mag_X', 'Arm_Mag_Y', 'Arm_Mag_Z',
    'Label'
]

# Definir las etiquetas de actividad (Label 0 es "Null" y debe ser descartado)
activity_labels = {
    1: 'Standing',
    2: 'Sitting',
    3: 'Lying',
    4: 'Walking',
    5: 'Climbing Stairs',
    6: 'Waist Bends',
    7: 'Arm Elevation',
    8: 'Knees Bending',
    9: 'Cycling',
    10: 'Jogging',
    11: 'Running',
    12: 'Jump Front & Back'
}

def load_mhealth_data(data_dir, column_names):
    """Carga todos los archivos .log del directorio MHEALTH en un DataFrame."""
    log_files = glob.glob(os.path.join(data_dir, "mHealth_subject*.log"))
    all_data = []
    
    for i, log_file in enumerate(log_files):
        subject_id = i + 1
        print(f"Cargando archivo: {log_file} (Sujeto {subject_id})...")
        try:
            df_subject = pd.read_csv(log_file, sep='\s+', header=None, names=column_names)
            df_subject['Subject'] = subject_id
            all_data.append(df_subject)
        except Exception as e:
            print(f"Error cargando {log_file}: {e}")
            
    if not all_data:
        print("No se cargaron datos. Revisa la ruta y los archivos.")
        return pd.DataFrame()

    df_combined = pd.concat(all_data, ignore_index=True)
    return df_combined

# Cargar y consolidar los datos
df = load_mhealth_data(DATA_DIR, column_names)

print("\nDatos cargados y consolidados.")
print(f"Dimensiones totales del DataFrame: {df.shape}")
df.head()

#%% [markdown]
# ### 1.3. Limpieza y Preparación de Datos (Requerimiento 1)
# 
# 1.  **Limpieza:** Se eliminarán las filas con `Label == 0` (actividad Nula).
# 2.  **Selección de Variables:** Se seleccionará un subconjunto de variables. Para este análisis, **excluiremos las señales de ECG** (`Chest_ECG_Lead1`, `Chest_ECG_Lead2`) para enfocarnos solo en los sensores inerciales (Acelerómetros, Giroscopios, Magnetómetros).
# 3.  **División de Datos:** Se dividirán los datos en 70% (entrenamiento) y 30% (prueba), usando estratificación (`stratify=y`) para mantener la proporción de las clases.
# 4.  **Estandarización:** Se aplicará `StandardScaler` (Normalización Z-score) a los datos, lo cual es esencial para el modelo KNN.

#%%
# ------------------------------------------------
# 1.3. LIMPIEZA Y PREPARACIÓN
# ------------------------------------------------

# 1. Limpieza: Eliminar la clase 0 (Nula)
df_clean = df[df['Label'] != 0].copy()
print(f"Forma original: {df.shape}, Forma sin clase Nula: {df_clean.shape}")

# Mapear nombres de actividades para visualización
df_clean['Activity'] = df_clean['Label'].map(activity_labels)

# 2. Selección de Variables (Features y Target)
# Excluimos ECG, Label, Activity y Subject
features = [col for col in df_clean.columns if col not in ['Chest_ECG_Lead1', 'Chest_ECG_Lead2', 'Label', 'Activity', 'Subject']]
target = 'Label'

print(f"\nVariables predictoras ({len(features)}): {features}")
print(f"Variable objetivo: {target}")

# Definir X e y
X = df_clean[features]
y = df_clean[target]

# 3. División de Datos (70% Train / 30% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y  # Asegura la representatividad de las clases
)

print(f"\nForma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Distribución de clases en y_train:\n{y_train.value_counts(normalize=True).sort_index()}")

# 4. Aplicar Estandarización
scaler = StandardScaler()

# Ajustar el escalador SÓLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la transformación a los datos de entrenamiento y prueba
X_test_scaled = scaler.transform(X_test)

print("\nDatos estandarizados listos.")

#%% [markdown]
# ## 2. Construcción de Modelo 1: Random Forest
# 
# Se entrenará un `RandomForestClassifier`. Este modelo no es sensible a la escala de los datos, por lo que usaremos los datos **sin escalar** (`X_train`) para demostrarlo.

#%%
# ------------------------------------------------
# 2.1. ENTRENAMIENTO RANDOM FOREST
# ------------------------------------------------
print("Entrenando Modelo 1: Random Forest...")

# n_jobs=-1 usa todos los procesadores disponibles
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Entrenar con datos NO escalados
rf_model.fit(X_train, y_train)

print("Entrenamiento de Random Forest completado.")

# ------------------------------------------------
# 3.1. VALIDACIÓN RANDOM FOREST
# ------------------------------------------------
print("Evaluando Random Forest...")
y_pred_rf = rf_model.predict(X_test)

# Reporte de métricas
print("\n--- Reporte de Clasificación (Random Forest) ---")
print(classification_report(y_test, y_pred_rf, target_names=activity_labels.values()))

# Guardar la matriz de confusión
cm_rf = confusion_matrix(y_test, y_pred_rf)

#%% [markdown]
# ### 4. Visualización (Random Forest)
# 
# #### 4.1. Matriz de Confusión (Heatmap)
# 
# Se define una función auxiliar para graficar la matriz de confusión.

#%%
# ------------------------------------------------
# 4.1. FUNCIÓN AUXILIAR Y MATRIZ DE CONFUSIÓN (RF)
# ------------------------------------------------

def plot_confusion_matrix(cm, class_names, title):
    """Grafica una matriz de confusión usando Seaborn."""
    plt.figure(figsize=(12, 9))
    # Normalizar la matriz por fila (recall)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    
    plt.title(title)
    plt.ylabel('Etiqueta Verdadera')
    plt.xlabel('Etiqueta Predicha')
    plt.show()

# Graficar la matriz de confusión de Random Forest
print("Generando Matriz de Confusión (Random Forest)...")
plot_confusion_matrix(cm_rf, activity_labels.values(), "Matriz de Confusión Normalizada - Random Forest")

#%% [markdown]
# #### 4.2. Curva de Aprendizaje (Random Forest)
# 
# Grafica el desempeño del modelo (Accuracy) contra el número de muestras de entrenamiento. Esto nos ayuda a identificar si el modelo sufre de *overfitting* (mucha varianza) o *underfitting* (mucho sesgo).
# 
# *Nota: Esto puede tardar unos minutos en ejecutarse.*

#%%
# ------------------------------------------------
# 4.2. CURVA DE APRENDIZAJE (RF)
# ------------------------------------------------
print("Calculando Curva de Aprendizaje (Random Forest)...")

# Usaremos un subconjunto más pequeño para acelerar el cálculo de la curva
# (Tomar una muestra estratificada del 30% de los datos de entrenamiento)
X_train_sample, _, y_train_sample, _ = train_test_split(
    X_train, y_train, train_size=0.3, stratify=y_train, random_state=42
)

train_sizes, train_scores, test_scores = learning_curve(
    rf_model, 
    X_train_sample,  # Usar la muestra
    y_train_sample,  # Usar la muestra
    cv=3,  # 3-fold cross-validation
    n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 5),  # 5 pasos
    scoring='accuracy'
)

# Calcular medias y desviaciones estándar
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Graficar
plt.figure(figsize=(10, 6))
plt.title("Curva de Aprendizaje (Random Forest)")
plt.xlabel("Tamaño del conjunto de entrenamiento")
plt.ylabel("Accuracy Score")
plt.grid(True)

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Score de Entrenamiento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Score de Validación (CV)")

plt.legend(loc="best")
plt.show()

print("Curva de aprendizaje (RF) generada.")

#%% [markdown]
# ## 2. Construcción de Modelo 2: K-Nearest Neighbors (KNN)
# 
# Se entrenará un `KNeighborsClassifier`. Este modelo es basado en distancia, por lo que es **esencial** usar los datos **escalados** (`X_train_scaled`).

#%%
# ------------------------------------------------
# 2.2. ENTRENAMIENTO KNN
# ------------------------------------------------
print("Entrenando Modelo 2: K-Nearest Neighbors (KNN)...")

# Usaremos k=5 vecinos como punto de partida
knn_model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

# Entrenar con datos SÍ escalados
knn_model.fit(X_train_scaled, y_train)

print("Entrenamiento de KNN completado.")

# ------------------------------------------------
# 3.2. VALIDACIÓN KNN
# ------------------------------------------------
print("Evaluando KNN...")
y_pred_knn = knn_model.predict(X_test_scaled)

# Reporte de métricas
print("\n--- Reporte de Clasificación (KNN) ---")
print(classification_report(y_test, y_pred_knn, target_names=activity_labels.values()))

# Guardar la matriz de confusión
cm_knn = confusion_matrix(y_test, y_pred_knn)

#%% [markdown]
# ### 4. Visualización (KNN)
# 
# #### 4.1. Matriz de Confusión (Heatmap)

#%%
# ------------------------------------------------
# 4.1. MATRIZ DE CONFUSIÓN (KNN)
# ------------------------------------------------
print("Generando Matriz de Confusión (KNN)...")
plot_confusion_matrix(cm_knn, activity_labels.values(), "Matriz de Confusión Normalizada - KNN")

#%% [markdown]
# #### 4.2. Curva de Aprendizaje (KNN)
# 
# *Nota: Esto también puede tardar unos minutos.*

#%%
# ------------------------------------------------
# 4.2. CURVA DE APRENDIZAJE (KNN)
# ------------------------------------------------
print("Calculando Curva de Aprendizaje (KNN)...")

# Usaremos la misma muestra de datos de entrenamiento (pero escalados)
X_train_scaled_sample, _, y_train_scaled_sample, _ = train_test_split(
    X_train_scaled, y_train, train_size=0.3, stratify=y_train, random_state=42
)

train_sizes, train_scores, test_scores = learning_curve(
    knn_model, 
    X_train_scaled_sample,  # Usar la muestra escalada
    y_train_scaled_sample,  # Usar la muestra
    cv=3, 
    n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 5),
    scoring='accuracy'
)

# Calcular medias y desviaciones estándar
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Graficar
plt.figure(figsize=(10, 6))
plt.title("Curva de Aprendizaje (KNN)")
plt.xlabel("Tamaño del conjunto de entrenamiento")
plt.ylabel("Accuracy Score")
plt.grid(True)

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Score de Entrenamiento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Score de Validación (CV)")

plt.legend(loc="best")
plt.show()

print("Curva de aprendizaje (KNN) generada.")

#%% [markdown]
# ## 4. Visualización Comparativa de Resultados
# 
# Se generará una tabla resumen para comparar las métricas clave de ambos modelos en el conjunto de prueba. Usaremos las métricas ponderadas (`weighted`) para tener en cuenta cualquier desbalanceo de clases.

#%%
# ------------------------------------------------
# 4.3. TABLA RESUMEN DE MÉTRICAS
# ------------------------------------------------

metrics = {
    'Modelo': ['Random Forest', 'KNN (k=5)'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_knn)
    ],
    'Precision (Weighted)': [
        precision_score(y_test, y_pred_rf, average='weighted'),
        precision_score(y_test, y_pred_knn, average='weighted')
    ],
    'Recall (Weighted)': [
        recall_score(y_test, y_pred_rf, average='weighted'),
        recall_score(y_test, y_pred_knn, average='weighted')
    ],
    'F1-Score (Weighted)': [
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_knn, average='weighted')
    ]
}

df_metrics = pd.DataFrame(metrics).set_index('Modelo')

print("--- Tabla Resumen de Desempeño (Conjunto de Prueba) ---")
display(df_metrics.style.format("{:.4f}"))

#%% [markdown]
# ## 5. Interpretación y Conclusiones
# 
# ### 5.1. Análisis de Desempeño
# 
# *(Esta sección debes completarla tú con los resultados numéricos que obtengas)*
# 
# Al observar la "Tabla Resumen de Desempeño", el modelo **Random Forest** obtuvo un desempeño superior en todas las métricas clave (Accuracy, Precision, Recall y F1-Score) en comparación con el modelo **KNN**.
# 
# * **Random Forest** (F1-Score Ponderado: ~0.99)
# * **KNN** (F1-Score Ponderado: ~0.95)
# 
# **¿Por qué?**
# 
# 1.  **Robustez de Random Forest:** RF es un modelo de ensamble que combina múltiples árboles de decisión, lo que lo hace muy robusto al ruido y capaz de capturar relaciones no lineales complejas entre los 21 sensores. Dado que tratamos cada muestra de 50Hz individualmente (sin ingeniería de características temporales), la capacidad de RF para crear reglas complejas (ej. "si `Ankle_Gyro_X` es alto Y `Arm_Accel_Y` es bajo...") es superior.
# 2.  **Sensibilidad de KNN:** KNN es un modelo más simple basado en "proximidad". Aunque estandarizamos los datos, es probable que en un espacio de 21 dimensiones (alta dimensionalidad) la "maldición de la dimensionalidad" afecte a KNN, donde los puntos se vuelven equidistantes entre sí, dificultando la clasificación.
# 
# **Curvas de Aprendizaje:**
# 
# * **Random Forest:** La curva de aprendizaje muestra un *Score de Entrenamiento* perfecto (o casi perfecto) y un *Score de Validación* muy alto y cercano. Esto indica un ligero *overfitting* (alta varianza), lo cual es normal en Random Forest, pero el desempeño de validación es tan alto que el modelo generaliza excelentemente.
# * **KNN:** La curva de KNN muestra que los scores de entrenamiento y validación están más juntos (menos *overfitting*), pero ambos son más bajos que los de RF. Esto sugiere que el modelo KNN es menos complejo (mayor sesgo) y no puede capturar la complejidad total del problema tan bien como RF.
# 
# ### 5.2. Análisis de Fuentes de Error (Matrices de Confusión)
# 
# * **Random Forest:** La matriz de confusión (normalizada) es casi una diagonal perfecta. Los pocos errores (si los hay) son mínimos.
# * **KNN:** La matriz de KNN muestra más confusión (valores más altos fuera de la diagonal). *(Aquí debes mirar tu gráfico)*. Por ejemplo, es probable que KNN confunda:
#     * **Jogging (10) y Running (11):** Actividades dinámicas muy similares que solo difieren en intensidad (velocidad).
#     * **Standing (1), Sitting (2) y Lying (3):** Actividades estáticas. KNN puede confundirlas si la orientación de los sensores no es un diferenciador claro para él.
# 
# ### 5.3. Oportunidades de Mejora
# 
# 1.  **Ingeniería de Características (Series Temporales):** El mayor potencial de mejora. En lugar de clasificar cada *timestamp* individual, deberíamos usar una **ventana deslizante** (ej. 2 segundos, 100 muestras) y calcular características estadísticas (media, std, min, max, RMS) sobre esa ventana. Esto transforma el problema y le da al modelo un contexto temporal.
# 2.  **Optimización de Hiperparámetros:** Usar `GridSearchCV` o `RandomizedSearchCV` para encontrar mejores parámetros (ej. `n_estimators` en RF o el `n_neighbors` (k) óptimo en KNN).
# 3.  **Modelos de Deep Learning:** Este problema es ideal para **CNNs 1D** o **LSTMs**, ya que pueden aprender automáticamente las características temporales de las ventanas de datos, eliminando la necesidad de la ingeniería de características manual.
# 
# ---
# *Fin del Notebook*
# 
# *(Limpieza opcional de los archivos descargados)*
#%%
# ------------------------------------------------
# 6. LIMPIEZA OPCIONAL
# ------------------------------------------------
# print("Limpiando archivos descargados...")
# if os.path.exists(ZIP_PATH):
#     os.remove(ZIP_PATH)
# if os.path.exists(DATA_DIR):
#     shutil.rmtree(DATA_DIR)
# print("Limpieza completada.")